

---


#**Hyperparameter Tuning for Lasso Regression without using lassoCV API**

---



In [2]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load the dataset
df = pd.read_csv('BostonHousing.csv')

In [3]:
# Preview the dataset (first 3 samples)
df.head(3)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7


In [4]:
# Split the data into features (X) and target (y)
X = df.iloc[:, :-1]  # All columns except the last one
y = df.iloc[:, -1]   # The last column

In [5]:
X

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48


In [6]:
y

,medv
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


In [7]:
# Initial Lasso Regression Model (without hyperparameter tuning)
model = Lasso()

In [8]:
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [9]:
# Train the model
model.fit(x_train, y_train)

Lasso()

In [10]:
# Predict on the test set
y_pred = model.predict(x_test)

In [11]:
# Evaluate the model using R² score
initial_r2 = r2_score(y_test, y_pred)
print(f"Initial R² Score: {initial_r2:.4f}")

Initial R² Score: 0.6622


###**If the model is not performing well, optimize it using hyperparameter tuning**

In [13]:
# Standardize the features
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [14]:
# Re-split the dataset into training and testing sets with standardized features
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [15]:
# Define the Lasso model and the hyperparameters grid
model_tuned = Lasso()
params = {'alpha': [0.00001, 0.0001, 0.001, 0.01]}

In [16]:
# Perform Grid Search with Cross-Validation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(model_tuned, params, cv=cv)
grid_search.fit(X_scaled, y)

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1),
             estimator=Lasso(),
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01]})

In [18]:
# Get the best hyperparameter from the grid search
best_alpha = grid_search.best_params_['alpha']
print(f"Best Alpha: {best_alpha}")

Best Alpha: 0.01


In [19]:
# Train the final model using the best hyperparameter
final_model = Lasso(alpha=best_alpha)
final_model.fit(x_train, y_train)

Lasso(alpha=0.01)

In [20]:
# Predict on the test set with the tuned model
y_pred_final = final_model.predict(x_test)

In [21]:
# Evaluate the final model using R² score
final_r2 = r2_score(y_test, y_pred_final)
print(f"Final R² Score: {final_r2:.4f}")

Final R² Score: 0.7787


##**Conclusion:**
###The final model shows an improvement in the R² score after tuning the alpha parameter.
